In [1]:
import json
import pickle
import requests
import sqlite3

from bs4 import BeautifulSoup
from movie_info import RatingDistribution, Media, Movie

In [2]:
headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)"}
imdb_url = 'https://www.imdb.com'

##### Определение вспомогательных функций

In [3]:
def get_main_tag(imdb_related_url, headers=headers):
    response = requests.get(imdb_related_url, headers=headers)
    soup = BeautifulSoup(response.text, 'lxml')
    body_tag = soup.head.next_sibling
    div_tag__next = body_tag.find('div', id='__next', recursive=False)
    return div_tag__next.find('main', recursive=False)

In [4]:
def remove_last_url_part(url):
    return url[:url.rfind('/')]

In [5]:
def save_object(obj, filename):
    with open(filename, 'wb') as outp:
        pickle.dump(obj, outp, pickle.HIGHEST_PROTOCOL)

##### Функция парсинга страницы с фильмом

In [6]:
def parse_movie_info(movie_page_url, headers=headers):
    main_tag = get_main_tag(movie_page_url, headers)
    section_outmost = main_tag.div.find('section', recursive=False)
    section_outer = section_outmost.find('section', recursive=False)
    genre_data = section_outer.find('div', {'data-testid': 'genres'})
    description_data = genre_data.next_sibling

    directors_data, writers_data, stars_data = description_data.next_sibling.find_all(
        'li', {'data-testid': 'title-pc-principal-credit'}
    )

    get_names = lambda data: (
        [item.text for item in list(data.children)[1].ul.find_all('li')]
    )
    directors = get_names(directors_data)
    writers = get_names(writers_data)
    stars = get_names(stars_data)
    
    genre_list_data = genre_data.find('div', class_='ipc-chip-list__scroller')
    genres = list(map(lambda s: s.text, genre_list_data.children))
    description = description_data.find('span', {'data-testid': 'plot-xl'}).text
    return (genres, description, directors, writers, stars)

In [7]:
test_movie_info = (
    parse_movie_info('https://www.imdb.com/title/tt0111161')
)
assert (test_movie_info == (
        ['Drama'],
        'Over the course of several years, two convicts form a friendship,'\
        ' seeking consolation and, eventually, redemption through basic compassion.',
        ['Frank Darabont'],
        ['Stephen King', 'Frank Darabont'],
        ['Tim Robbins', 'Morgan Freeman', 'Bob Gunton']
    )
)
test_movie_info

(['Drama'],
 'Over the course of several years, two convicts form a friendship, seeking consolation and, eventually, redemption through basic compassion.',
 ['Frank Darabont'],
 ['Stephen King', 'Frank Darabont'],
 ['Tim Robbins', 'Morgan Freeman', 'Bob Gunton'])

##### Функция парсинга детализированного рейтинга

In [8]:
def parse_detailed_ratings(ratings_page, headers=headers):
    response = requests.get(ratings_page, headers=headers)
    soup = BeautifulSoup(response.text, 'lxml')

    script = soup.head.next_sibling.find('script', id='__NEXT_DATA__', recursive=False)
    if not script:
        raise
    script_text = script.text
    score_data_pos = script_text.find('histogramValues')
    if score_data_pos == -1:
        raise

    unneeded_data_pos = script_text[score_data_pos:].find('countryData')
    if unneeded_data_pos == -1:
        raise
    narrowed_data = script_text[score_data_pos: score_data_pos + unneeded_data_pos]
    start_data_pos = narrowed_data.find('[')
    end_data_pos = narrowed_data.rfind(']') + 1
    list_of_vote_info = json.loads(narrowed_data[start_data_pos:end_data_pos])

    rating_dist = RatingDistribution()
    for i in range(10):
        cur_vote_count = int(list_of_vote_info[i]['voteCount'])
        rating_dist[i + 1] = cur_vote_count
    return rating_dist

In [9]:
test_movie_rating_dist = parse_detailed_ratings('https://www.imdb.com/title/tt0111161/ratings')
assert isinstance(test_movie_rating_dist, RatingDistribution)
assert len(test_movie_rating_dist) == 10
test_movie_rating_dist

{1: 41912,
 2: 5267,
 3: 5733,
 4: 7809,
 5: 16299,
 6: 31544,
 7: 105498,
 8: 333308,
 9: 730016,
 10: 1557486}

#### Парсинг фильмов

In [10]:
movies_href = '/chart/top'
movies_url = imdb_url + movies_href
movies_response = requests.get(movies_url, headers=headers)
movies_soup = BeautifulSoup(movies_response.text, 'lxml')

In [11]:
movies_iterator = movies_soup.find('ul', class_='ipc-metadata-list').children

In [12]:
movie_list = list(movies_iterator)
assert len(movie_list) == 250

In [13]:
movies_res = []
for movie_data in movie_list:
    a_title_tag = movie_data.find('a', class_='ipc-title-link-wrapper')
    movie_metadata = movie_data.find('div', class_='cli-title-metadata')
    rating_data = movie_metadata.next_sibling

    _, *rating_and_votes = rating_data.find('span').children
    user_rating = rating_and_votes[0].text
    str_vote_count = rating_and_votes[1].text[2:-1]

    metadata_text = tuple(map(lambda s: s.text, movie_metadata.children))
    if len(metadata_text) == 2:
        year_str, duration_str = metadata_text
        restriction_rating = None
    elif len(metadata_text) == 3:
        year_str, duration_str, restriction_rating = metadata_text
    else:
        raise

    if a_title_tag['href'] is None:
        raise

    movie_href = remove_last_url_part(a_title_tag['href'])
    rank_and_title = a_title_tag.text
    rank, title = rank_and_title.split('.', 1)
    rank = int(rank)
    title = title.strip()
    
    movie_page = imdb_url + movie_href
    genres, description, directors, writers, stars = parse_movie_info(movie_page)
    rating_dist = parse_detailed_ratings(movie_page + '/ratings')
    hours_and_minutes = duration_str.split(' ')
    hours_str = hours_and_minutes[0]
    hours = int(hours_str[:hours_str.find('h')])
    minutes = 0
    if len(hours_and_minutes) == 2:
        minutes_str = hours_and_minutes[1]
        minutes = int(minutes_str[:minutes_str.find('m')])
    year = int(year_str)
    duration = hours * 60 + minutes
    movie = Movie(rank=rank, title=title, rating_str=user_rating,
              vote_count=rating_dist.total_vote_count,
              directors=directors, writers=writers, stars=stars,
              year=year, genres=genres, description=description,
              rating_dist=rating_dist, duration=duration)
    movies_res.append(movie)
    print(movie, end='\n\n')

rank = 1
title = Побег из Шоушенка
rating = 9.3
vote count = 2834872
director(s) = ['Frank Darabont']
writer(s) = ['Stephen King', 'Frank Darabont']
star(s) = ['Tim Robbins', 'Morgan Freeman', 'Bob Gunton']
year = 1994
genres = ['Drama']
description = Over the course of several years, two convicts form a friendship, seeking consolation and, eventually, redemption through basic compassion.
duration = 142

rank = 2
title = Крёстный отец
rating = 9.2
vote count = 1975731
director(s) = ['Francis Ford Coppola']
writer(s) = ['Mario Puzo', 'Francis Ford Coppola']
star(s) = ['Marlon Brando', 'Al Pacino', 'James Caan']
year = 1972
genres = ['Crime', 'Drama']
description = Don Vito Corleone, head of a mafia family, decides to hand over his empire to his youngest son Michael. However, his decision unintentionally puts the lives of his loved ones in grave danger.
duration = 175

rank = 3
title = Тёмный рыцарь
rating = 9.0
vote count = 2816161
director(s) = ['Christopher Nolan']
writer(s) = ['Jonat

In [14]:
for movie in movies_res[:10]:
    print(movie.rating_dist)

{1: 41912, 2: 5267, 3: 5733, 4: 7809, 5: 16299, 6: 31544, 7: 105498, 8: 333308, 9: 730016, 10: 1557486}
{1: 41674, 2: 6940, 3: 7517, 4: 10168, 5: 20005, 6: 36366, 7: 96614, 8: 245765, 9: 491483, 10: 1019199}
{1: 39813, 2: 8375, 3: 9212, 4: 12952, 5: 26678, 6: 56512, 7: 161052, 8: 425502, 9: 797123, 10: 1278942}
{1: 22875, 2: 4542, 3: 5130, 4: 7255, 5: 15126, 6: 30895, 7: 83492, 8: 208343, 9: 385302, 10: 577975}
{1: 9869, 2: 1807, 3: 2150, 4: 3457, 5: 7585, 6: 18347, 7: 57016, 8: 159799, 9: 272715, 10: 313183}
{1: 22002, 2: 4880, 3: 5245, 4: 7100, 5: 14947, 6: 30823, 7: 94246, 8: 256254, 9: 450325, 10: 538626}
{1: 32815, 2: 7324, 3: 8964, 4: 12188, 5: 24983, 6: 50977, 7: 134504, 8: 313592, 9: 510324, 10: 845579}
{1: 29744, 2: 10252, 3: 12010, 4: 16647, 5: 31705, 6: 62983, 7: 173121, 8: 407690, 9: 647734, 10: 782534}
{1: 30492, 2: 7758, 3: 9353, 4: 12534, 5: 26036, 6: 54257, 7: 155284, 8: 372826, 9: 544105, 10: 755707}
{1: 11497, 2: 2352, 3: 2934, 4: 4613, 5: 10142, 6: 22246, 7: 69996, 8

In [15]:
save_object(movies_res, 'movie_list.pkl')

#### Создание таблиц

In [16]:
conn = sqlite3.connect('movie_db')

In [17]:
sql_create_movie_table = '''
    CREATE TABLE IF NOT EXISTS Movie (
        id INTEGER PRIMARY KEY,
        rank INTEGER UNIQUE,
        rating REAL NOT NULL,
        vote_count INTEGER NOT NULL,
        year INTEGER NOT NULL,
        title TEXT NOT NULL,
        description TEXT,
        duration INTEGER NOT NULL
    );'''

In [18]:
sql_create_person_table = '''
    CREATE TABLE IF NOT EXISTS Person (
        id INTEGER PRIMARY KEY,
        name TEXT UNIQUE NOT NULL
    );'''

In [19]:
sql_create_genre_table = '''
    CREATE TABLE IF NOT EXISTS Genre (
        id INTEGER PRIMARY KEY,
        name TEXT UNIQUE NOT NULL
    );'''

In [20]:
sql_create_movie_director_table = '''
    CREATE TABLE IF NOT EXISTS MovieDirector (
        movie_id INTEGER NOT NULL,
        director_id INTEGER NOT NULL,
        UNIQUE(movie_id, director_id),
        FOREIGN KEY(movie_id) REFERENCES Movie(id),
        FOREIGN KEY(director_id) REFERENCES Person(id)
    );'''

In [21]:
sql_create_movie_writer_table = '''
    CREATE TABLE IF NOT EXISTS MovieWriter (
        movie_id INTEGER NOT NULL,
        writer_id INTEGER NOT NULL,
        UNIQUE(movie_id, writer_id),
        FOREIGN KEY(movie_id) REFERENCES Movie(id),
        FOREIGN KEY(writer_id) REFERENCES Person(id)
    );'''

In [22]:
sql_create_movie_star_table = '''
    CREATE TABLE IF NOT EXISTS MovieStar (
        movie_id INTEGER NOT NULL,
        star_id INTEGER NOT NULL,
        UNIQUE(movie_id, star_id),
        FOREIGN KEY(movie_id) REFERENCES Movie(id),
        FOREIGN KEY(star_id) REFERENCES Person(id)
    );'''

In [23]:
sql_create_movie_genre_table = '''
    CREATE TABLE IF NOT EXISTS MovieGenre (
        movie_id INTEGER NOT NULL,
        genre_id INTEGER NOT NULL,
        UNIQUE(movie_id, genre_id),
        FOREIGN KEY(movie_id) REFERENCES Movie(id),
        FOREIGN KEY(genre_id) REFERENCES Genre(id)
    );'''

In [24]:
sql_create_movie_rating_dist_table = '''
    CREATE TABLE IF NOT EXISTS MovieRatingDist (
        movie_id INTEGER NOT NULL,
        rating INTEGER,
        vote_count INTEGER NOT NULL,
        UNIQUE(movie_id, rating)
    );'''

In [25]:
cursor = conn.cursor()

In [26]:
cursor.execute(sql_create_movie_table)
cursor.execute(sql_create_person_table)
cursor.execute(sql_create_genre_table)
cursor.execute(sql_create_movie_director_table)
cursor.execute(sql_create_movie_writer_table)
cursor.execute(sql_create_movie_star_table)
cursor.execute(sql_create_movie_genre_table)
cursor.execute(sql_create_movie_rating_dist_table)

In [27]:
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
print(cursor.fetchall())

[('Movie',), ('Person',), ('Genre',), ('MovieDirector',), ('MovieWriter',), ('MovieStar',), ('MovieGenre',), ('MovieRatingDist',)]


#### Заполнение таблиц

In [28]:
genre_set = set()
genre_list = []

person_set = set()
person_list = []

In [29]:
for movie in movies_res:
    for d in movie.directors:
        if d not in person_set:
            person_set.add(d)
            person_list.append(d)
    for w in movie.writers:
        if w not in person_set:
            person_set.add(w)
            person_list.append(w)
    for s in movie.stars:
        if s not in person_set:
            person_set.add(s)
            person_list.append(s)
    for g in movie.genres:
        if g not in genre_set:
            genre_set.add(g)
            genre_list.append(g)

In [30]:
person_list

['Frank Darabont',
 'Stephen King',
 'Tim Robbins',
 'Morgan Freeman',
 'Bob Gunton',
 'Francis Ford Coppola',
 'Mario Puzo',
 'Marlon Brando',
 'Al Pacino',
 'James Caan',
 'Christopher Nolan',
 'Jonathan Nolan',
 'David S. Goyer',
 'Christian Bale',
 'Heath Ledger',
 'Aaron Eckhart',
 'Robert De Niro',
 'Robert Duvall',
 'Sidney Lumet',
 'Reginald Rose',
 'Henry Fonda',
 'Lee J. Cobb',
 'Martin Balsam',
 'Steven Spielberg',
 'Thomas Keneally',
 'Steven Zaillian',
 'Liam Neeson',
 'Ralph Fiennes',
 'Ben Kingsley',
 'Peter Jackson',
 'J.R.R. Tolkien',
 'Fran Walsh',
 'Philippa Boyens',
 'Elijah Wood',
 'Viggo Mortensen',
 'Ian McKellen',
 'Quentin Tarantino',
 'Roger Avary',
 'John Travolta',
 'Uma Thurman',
 'Samuel L. Jackson',
 'Orlando Bloom',
 'Sergio Leone',
 'Luciano Vincenzoni',
 'Agenore Incrocci',
 'Clint Eastwood',
 'Eli Wallach',
 'Lee Van Cleef',
 'Robert Zemeckis',
 'Winston Groom',
 'Eric Roth',
 'Tom Hanks',
 'Robin Wright',
 'Gary Sinise',
 'David Fincher',
 'Chuck Pal

##### Заполнение таблицы Person

In [32]:
for person in person_list:
    cursor.execute("INSERT INTO Person(name) VALUES (?);", (person,))

In [33]:
cursor.execute("SELECT * FROM Person;")
person_rows = cursor.fetchall()

In [34]:
print(person_rows[:20])

[(1, 'Frank Darabont'), (2, 'Stephen King'), (3, 'Tim Robbins'), (4, 'Morgan Freeman'), (5, 'Bob Gunton'), (6, 'Francis Ford Coppola'), (7, 'Mario Puzo'), (8, 'Marlon Brando'), (9, 'Al Pacino'), (10, 'James Caan'), (11, 'Christopher Nolan'), (12, 'Jonathan Nolan'), (13, 'David S. Goyer'), (14, 'Christian Bale'), (15, 'Heath Ledger'), (16, 'Aaron Eckhart'), (17, 'Robert De Niro'), (18, 'Robert Duvall'), (19, 'Sidney Lumet'), (20, 'Reginald Rose')]


In [35]:
person_to_id = dict()
for person in person_rows:
    person_to_id[person[1]] = person[0]

##### Заполнение таблицы Genre

In [36]:
for genre in genre_list:
    cursor.execute("INSERT INTO Genre(name) VALUES (?);", (genre,))

In [37]:
cursor.execute("SELECT * FROM Genre;")
genre_rows = cursor.fetchall()

In [38]:
print(genre_rows)

[(1, 'Drama'), (2, 'Crime'), (3, 'Action'), (4, 'Biography'), (5, 'History'), (6, 'Adventure'), (7, 'Western'), (8, 'Romance'), (9, 'Sci-Fi'), (10, 'Fantasy'), (11, 'Mystery'), (12, 'Family'), (13, 'Thriller'), (14, 'War'), (15, 'Comedy'), (16, 'Animation'), (17, 'Music'), (18, 'Horror'), (19, 'Film-Noir'), (20, 'Musical'), (21, 'Sport')]


In [39]:
genre_to_id = dict()
for genre in genre_rows:
    genre_to_id[genre[1]] = genre[0]

##### Заполнение таблицы Movie

In [40]:
for movie in movies_res:
    cursor.execute('''
                   INSERT INTO Movie(rank, rating, vote_count,
                                     year, title, description, duration)
                   VALUES (?, ?, ?, ?, ?, ?, ?);''',
                   (movie.rank, movie.rating_str, movie.vote_count,
                   movie.year, movie.title, movie.description, movie.duration)
                  )

##### Заполнение связующих таблиц

In [41]:
for movie in movies_res:
    for d in movie.directors:        
        cursor.execute('''INSERT INTO MovieDirector(movie_id, director_id) VALUES (?, ?);''',
                (movie.rank, person_to_id[d]))
    for w in movie.writers:        
        cursor.execute('''INSERT INTO MovieWriter(movie_id, writer_id) VALUES (?, ?);''',
                (movie.rank, person_to_id[w]))
    for s in movie.stars:        
        cursor.execute('''INSERT INTO MovieStar(movie_id, star_id) VALUES (?, ?);''',
                (movie.rank, person_to_id[s]))
    for g in movie.genres:        
        cursor.execute('''INSERT INTO MovieGenre(movie_id, genre_id) VALUES (?, ?);''',
                (movie.rank, genre_to_id[g]))
    for vote, vote_count in movie.rating_dist.items():
        cursor.execute('''INSERT INTO MovieRatingDist VALUES (?, ?, ?);''',
                (movie.rank, vote, vote_count))

In [42]:
conn.commit()